# Extract Single Lap for Upload

This notebook extracts a single lap from the cleaned telemetry data and exports it as a CSV file that can be uploaded to the dashboard for comparison.

**Required CSV columns for upload:**
- `timestamp` (ISO 8601 format)
- `speed`
- `Steering_Angle`
- `pbrake_f` (front brake pressure)
- `pbrake_r` (rear brake pressure)
- `aps` or `ath` (throttle)

In [1]:
import os
import pandas as pd
import re

## Configuration

Set these parameters to select which lap to extract:

In [2]:
# ===== CONFIGURATION =====
# Choose the track folder (e.g., 'barber1', 'cota1', 'indy1', etc.)
TRACK_FOLDER = 'cota1'

# Choose the driver number
DRIVER_NUMBER = 7

# Choose the lap number
LAP_NUMBER = 5

# Output filename
OUTPUT_FILENAME = f'lap_{DRIVER_NUMBER}_{LAP_NUMBER}_{TRACK_FOLDER}.csv'
# =========================

## Load Telemetry Data

In [3]:
# Path to the telemetry file
telemetry_path = f'../datasets_clean/{TRACK_FOLDER}/telemetry_per_timestamp.csv'

if not os.path.exists(telemetry_path):
    print(f"❌ Error: File not found: {telemetry_path}")
    print(f"\nAvailable track folders:")
    for folder in os.listdir('../datasets_clean'):
        if os.path.isdir(f'../datasets_clean/{folder}'):
            print(f"  - {folder}")
else:
    print(f"✅ Loading telemetry from: {telemetry_path}")
    df = pd.read_csv(telemetry_path)
    print(f"📊 Loaded {len(df):,} total telemetry points")
    print(f"\nColumns: {list(df.columns)}")

✅ Loading telemetry from: ../datasets_clean/cota1/telemetry_per_timestamp.csv
📊 Loaded 1,059,861 total telemetry points

Columns: ['vehicle_id', 'meta_time', 'timestamp', 'lap', 'Steering_Angle', 'accx_can', 'accy_can', 'ath', 'gear', 'nmot', 'pbrake_f', 'pbrake_r', 'speed', 'NUMBER']


## Extract Driver Number and Filter Lap

In [4]:
# Extract numeric driver number from vehicle_id
df['NUMBER'] = df['vehicle_id'].apply(
    lambda x: int(re.search(r'-(\d+)$', str(x)).group(1)) if re.search(r'-(\d+)$', str(x)) else None
)

# Convert lap to numeric
df['lap'] = pd.to_numeric(df['lap'], errors='coerce')

# Filter for the specific driver and lap
lap_df = df[(df['NUMBER'] == DRIVER_NUMBER) & (df['lap'] == LAP_NUMBER)].copy()

print(f"🏁 Found {len(lap_df):,} telemetry points for Driver #{DRIVER_NUMBER}, Lap {LAP_NUMBER}")

if len(lap_df) == 0:
    print(f"\n❌ No data found for this driver/lap combination.")
    print(f"\nAvailable drivers in {TRACK_FOLDER}:")
    print(df['NUMBER'].dropna().unique())
    print(f"\nAvailable laps for driver {DRIVER_NUMBER}:")
    driver_laps = df[df['NUMBER'] == DRIVER_NUMBER]['lap'].dropna().unique()
    print(sorted(driver_laps))

🏁 Found 3,290 telemetry points for Driver #7, Lap 5


## Prepare Upload CSV

The dashboard requires specific columns. We'll ensure all required columns are present.

In [5]:
if len(lap_df) > 0:
    # Required columns for upload
    required_cols = ['timestamp', 'speed', 'Steering_Angle', 'pbrake_f', 'pbrake_r']
    
    # Add throttle column (aps or ath, whichever exists)
    if 'aps' in lap_df.columns:
        required_cols.append('aps')
        throttle_col = 'aps'
    elif 'ath' in lap_df.columns:
        # If only ath exists, rename it to aps for compatibility
        lap_df['aps'] = lap_df['ath']
        required_cols.append('aps')
        throttle_col = 'ath (renamed to aps)'
    else:
        print("⚠️ Warning: No throttle column (aps or ath) found")
        throttle_col = 'none'
    
    # Optional but useful columns
    optional_cols = ['gear', 'nmot', 'accx_can', 'accy_can', 'VBOX_Lat_Min', 'VBOX_Long_Minutes']
    
    # Select columns that exist
    export_cols = [col for col in required_cols if col in lap_df.columns]
    export_cols += [col for col in optional_cols if col in lap_df.columns]
    
    # Create export dataframe
    export_df = lap_df[export_cols].copy()
    
    # Check data quality
    print(f"\n📋 Export Summary:")
    print(f"  Total points: {len(export_df):,}")
    print(f"  Columns: {len(export_cols)}")
    print(f"  Throttle column: {throttle_col}")
    print(f"\n📊 Data Quality Check:")
    for col in required_cols:
        if col in export_df.columns:
            non_null = export_df[col].notna().sum()
            pct = (non_null / len(export_df)) * 100
            print(f"  {col}: {non_null:,} / {len(export_df):,} ({pct:.1f}%) non-null")
    
    # Show sample data
    print(f"\n🔍 Sample data (first 5 rows):")
    display(export_df.head())


📋 Export Summary:
  Total points: 3,290
  Columns: 10
  Throttle column: ath (renamed to aps)

📊 Data Quality Check:
  timestamp: 3,290 / 3,290 (100.0%) non-null
  speed: 678 / 3,290 (20.6%) non-null
  Steering_Angle: 3,196 / 3,290 (97.1%) non-null
  pbrake_f: 3,196 / 3,290 (97.1%) non-null
  pbrake_r: 3,196 / 3,290 (97.1%) non-null
  aps: 3,196 / 3,290 (97.1%) non-null

🔍 Sample data (first 5 rows):


,timestamp,speed,Steering_Angle,pbrake_f,pbrake_r,aps,gear,nmot,accx_can,accy_can
44839,2025-04-24 18:38:58.166000+00:00,NaN,5.2,0.0,0.0,100.00,4.0,NaN,0.136,-0.027
44840,2025-04-24 18:38:58.211000+00:00,NaN,5.2,0.0,0.0,99.99,4.0,NaN,0.114,-0.027
44841,2025-04-24 18:38:58.255000+00:00,171.18,5.2,0.0,0.0,100.02,4.0,6722.0,0.127,-0.019
44842,2025-04-24 18:38:58.300000+00:00,NaN,5.1,0.0,0.0,99.98,4.0,NaN,0.125,-0.033
44843,2025-04-24 18:38:58.344000+00:00,NaN,4.9,0.0,0.0,99.99,4.0,NaN,0.153,-0.047


## Export to CSV

In [6]:
if len(lap_df) > 0:
    # Create output directory if it doesn't exist
    output_dir = '../lap_exports'
    os.makedirs(output_dir, exist_ok=True)
    
    # Full output path
    output_path = os.path.join(output_dir, OUTPUT_FILENAME)
    
    # Export to CSV
    export_df.to_csv(output_path, index=False)
    
    # Get file size
    file_size = os.path.getsize(output_path)
    file_size_mb = file_size / (1024 * 1024)
    
    print(f"\n✅ Successfully exported lap data!")
    print(f"📁 File: {output_path}")
    print(f"📦 Size: {file_size_mb:.2f} MB ({file_size:,} bytes)")
    print(f"\n🎯 You can now upload this CSV file to the dashboard for comparison!")
else:
    print("\n❌ Cannot export: No data found for the specified driver/lap.")


✅ Successfully exported lap data!
📁 File: ../lap_exports\lap_7_5_cota1.csv
📦 Size: 0.24 MB (253,997 bytes)

🎯 You can now upload this CSV file to the dashboard for comparison!


## Validation Check

Verify the exported file meets upload requirements:

In [7]:
if len(lap_df) > 0:
    # Reload the exported file to verify
    verify_df = pd.read_csv(output_path)
    
    print("🔍 Validation Check:")
    print(f"\n✓ Required columns:")
    
    required_for_upload = ['timestamp', 'speed', 'Steering_Angle', 'pbrake_f', 'pbrake_r', 'aps']
    all_present = True
    
    for col in required_for_upload:
        if col in verify_df.columns:
            print(f"  ✅ {col}")
        else:
            print(f"  ❌ {col} - MISSING")
            all_present = False
    
    if all_present:
        print(f"\n✅ All required columns present! File is ready for upload.")
    else:
        print(f"\n⚠️ Some required columns are missing. Upload may fail.")
    
    # Check timestamp format
    sample_timestamp = verify_df['timestamp'].iloc[0]
    print(f"\n📅 Sample timestamp: {sample_timestamp}")
    
    try:
        pd.to_datetime(sample_timestamp)
        print(f"  ✅ Timestamp format is valid")
    except:
        print(f"  ⚠️ Timestamp format may not be valid ISO 8601")

🔍 Validation Check:

✓ Required columns:
  ✅ timestamp
  ✅ speed
  ✅ Steering_Angle
  ✅ pbrake_f
  ✅ pbrake_r
  ✅ aps

✅ All required columns present! File is ready for upload.

📅 Sample timestamp: 2025-04-24 18:38:58.166000+00:00
  ✅ Timestamp format is valid
